In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 19)

### Tables in the process

In [2]:
format_dict = {
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'SHARES':'{:,}',    
    'Q4':'{:.4f}','Q3':'{:.4f}','Q2':'{:.4f}','Q1':'{:.4f}','DIVIDEND':'{:.4f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
}

In [19]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,RCL,192,2021-11-08,"1,500",44.50,908,1042,2022-01-19,49.50,153.70,10.76,"74,085.54",72,"7,187.70",10.74%,54.47%,92,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [23]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)

sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1042



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,RCL,1042,192,2021-11-08,"1,500",44.50,138.17,9.67,Sold,"66,897.84",HD,None


In [21]:
transaction, name, qty, sell_price, buy_price

('S', 'RCL', -1500, 49.5, 44.5)

### In case of buy, retreive buy id from latest buy record (id)

In [22]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,SCC,1103,86,2022-01-18,300,384.0,238.46,16.69,Active,115455.15,DTD,None


In [53]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'SCC', 300, 384.0)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [24]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'RCL'



,name,date,shares,price,active,dividend,period,grade
0,RCL,2021-11-08,"6,000",44.50,1,3.5000,3,A4


In [25]:
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -1500
WHERE name = 'RCL'



In [26]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [27]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'RCL'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,RCL,2021-11-08,4500.0,44.5,0.0,0.0,1,3.5,3,A4


### If shares become zero and no longer wants to keep buy record

In [18]:
sqlDel = """
DELETE FROM buy
WHERE name = '%s' AND volbuy = 0
"""
sqlDel = sqlDel % name
print(sqlDel)


DELETE FROM buy
WHERE name = 'SAT' AND volbuy = 0



In [19]:
rp = const.execute(sqlDel)
rp.rowcount

1

### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [28]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'RCL'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,RCL,0.5000,1.5000,1.5000,0.0000,3.5000,"6,000",2021-12-03,2021-12-03,,0


In [29]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -1500
WHERE name = 'RCL'



In [30]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [31]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'RCL'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,RCL,0.5000,1.5000,1.5000,0.0000,3.5000,"4,500",2021-12-03,2021-12-03,,0


### If shares become zero and no longer wants to keep dividend record

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [32]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'RCL'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,662,RCL,67.75,17.40,I,47.00,53.50,517.59,0.63,44.50,"1,500",-16,26,"6,000",0,0,20pct,SET100


In [35]:
buy_target  = 46.50   # must calculate manually beforehand
sell_target = 49.00
reason = "10pct"
sell_qty = 1500

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, reason = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, reason, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -1500,
qty = 1500, buy_target = 46.5, sell_target = 49.0, reason = '10pct'
WHERE name = 'RCL'



In [36]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [37]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'RCL'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,662,RCL,67.75,17.40,I,46.50,49.00,517.59,0.63,44.50,"1,500",-16,26,"4,500",0,0,10pct,SET100


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [54]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'SCC'



,name,date,shares,price,active,dividend,period,grade
0,SCC,2021-09-20,600,405.00,1,9.9999,3,A2


### In case there is the same stock name in port

In [55]:
new_cost = 398 # new average cost must be calculated manually beforehand

sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 300, price = 398
WHERE name = 'SCC'



In [56]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [57]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'SCC'



,name,date,shares,price,active,dividend,period,grade
0,SCC,2021-09-20,900,398.00,1,9.9999,3,A2


### In case of totally new stock

In [ ]:
sqlIns = """
INSERT INTO buy 
VALUES('LPF','2022-01-14',10000,14,0,0,1,0,'4','A3')
"""
print(sqlIns)

In [ ]:
#rp = const.execute(sqlIns)
rp.rowcount

### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [58]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'SCC' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,SCC,8.5000,0.0000,8.5000,0.0000,17.0000,600,2021-08-27,2021-08-27,,0


In [59]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 300
WHERE name = 'SCC'



In [60]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [61]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'SCC' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,SCC,8.5000,0.0000,8.5000,0.0000,17.0000,900,2021-08-27,2021-08-27,,0


### End of Update dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [62]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)


SELECT * 
FROM stocks 
WHERE name = 'SCC'



In [63]:
stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,578,SCC,474.00,364.00,B,384.00,0.00,"1,022.91",0.66,405.00,300,-17,16,600,0,12,RD05pct,SET50


In [64]:
new_cost = 398
buy_target = 378
sell_target = 418
status = 'B'
reason = 'RD05pct'
new_buy_qty = 300
new_sell_qty = 300
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

(398, 378, 418, 'B', 'RD05pct', 300, 300)

In [65]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 300, 
cost = 398, buy_target = 378, sell_target = 418, status = 'B', 
reason = 'RD05pct', qty = 300
WHERE name = 'SCC'



In [66]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [67]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'SCC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,578,SCC,474.00,364.00,B,378.00,418.00,"1,022.91",0.66,398.00,300,-17,16,900,0,12,RD05pct,SET50


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process